In [5]:
using DelimitedFiles
using Plots
using JLD
using GraphRecipes
using Statistics
push!(LOAD_PATH, pwd())
using FuncionesEEG
# import Pkg; Pkg.add("Combinatorics")
# import Pkg; Pkg.add("StatsBase")

In [4]:
import Pkg; Pkg.status()

      Status `C:\Users\reyal\.julia\environments\v1.7\Project.toml`
  [861a8166] Combinatorics v1.0.2
  [bd48cda9] GraphRecipes v0.5.12
  [7073ff75] IJulia v1.24.0
  [4138dd39] JLD v0.13.3
  [91a5bcdd] Plots v1.38.9
  [2913bbd2] StatsBase v0.33.21


## Resultados

In [ ]:
# Aquí se da la dirección donde está el archivo a trabajar
#Direccion = "C:/Users/1100423746/Dropbox/EEG/exp136_Igancio_32CH.dat"
Direccion = "exp136_Igancio_32CH.dat";
Dir = split(Direccion,".")[1];
Archivo = split(basename(Direccion),".")[1];
Datos = readdlm( Direccion );
ruta_resultados = joinpath(Dir * "_Resultados" );
mkpath( ruta_resultados );
#-----------------------------------------------------------------------------------------#
Headers , Canales = DatosDeTodos( Datos , 2 , 3 );
NomCan = Archivo*"_Canales.jld"
ruta_archivo = joinpath( ruta_resultados , NomCan )
save( ruta_archivo , Canales )
#-------------------------------------------------------------------------------------------#
# Funcion necesaria para encontrar los vecinos
elementos = [ -1 0 1 ]
combs = COMS(elementos);
lados = [ 3 5 5 5 5 5 2 ]
MVCuadComp = ArrayParaHeatmap( lados , Canales , Prohibidas , Coordenadas );
NomCuad = Archivo*"_DatosEnCuadrado.jld"
ruta_archivo = joinpath( ruta_resultados , NomCuad )
save( ruta_archivo , "MVCuacComp" , MVCuadComp )
MVYPromedios = PromVacios(Prohibidas,Coordenadas,combs,Headers,Canales,MVCuadComp);
#-------------------------------------------------------------------------------------------#
MVcsda = CSDAZapfe( MVYPromedios );
NomCSDA = Archivo*"_CSDA.jld"
ruta_archivo = joinpath( ruta_resultados , NomCSDA )
save( ruta_archivo , "MVcsda" , MVcsda )
#-------------------------------------------------------------------------------------------#
CMP , CMN = CentrosDeMasa( MVcsda , MVcsda , 0.5 );
NomCM = Archivo*"_CM.jld"
ruta_archivo = joinpath( ruta_resultados , NomCM )
save( ruta_archivo , "CMP" , CMP , "CMN" , CMN )
#-------------------------------------------------------------------------------------------#
DistTol = 1.5
TTol = 3
TraysPos = Dinamica( CMP , DistTol , TTol );
TraysNeg = Dinamica( CMN , DistTol , TTol );
NomTrays = Archivo*"_Trayectorias.jld"
ruta_archivo = joinpath( ruta_resultados , NomTrays )
save( ruta_archivo , "TP" , TraysPos , "TN" , TraysNeg )
#-------------------------------------------------------------------------------------------#
# El siguiente es un diccionario que contiene el inicio y final de cada trayectoria
( IniciosP , FinalesP ) = IniciosYFinales( TraysPos );
( IniciosN , FinalesN ) = IniciosYFinales( TraysNeg );
# Ahora rellenamos los huecos que existan en las trayectorias
TraysPos = RellenaHuecos( TraysPos , IniciosP , FinalesP );
TraysNeg = RellenaHuecos( TraysNeg , IniciosN , FinalesN );
# En esta sección recorremos las coordenadas y las transformamos a polares
TraysPosP = ArreglaCoordenadas( TraysPos , lados );
TraysNegP = ArreglaCoordenadas( TraysNeg , lados );
NomTPs = Archivo*"_TraysPolares.jld"
ruta_archivo = joinpath( ruta_resultados , NomTPs )
save( ruta_archivo , "TPP" , TraysPosP , "TNP" , TraysNegP )

In [ ]:
# Debido a que la frecuecia es 500 Hz, debe haber 500 frames por un segundo

In [3]:
# Generamos colores para las trayectorias para evitar que se confundan
CTPs = [ :red1 , :lime , :orange, :cyan , :grey54 , :blue2 , :green2 , :deeppink ];
CTP = GeneraColores( CTPs , TraysPosP );
CTNs = [ :wheat3 , :teal , :plum, :bisque , :gray0 , :navy , :pink3 , :indigo ];
CTN = GeneraColores( CTNs , TraysNegP );

## Gifs

In [4]:
cachos = 15
tamaño = size( MVcsda )[ 3 ] / cachos
nomcsda = "CSDA_"
nombintray = "Trayectorias_"
nd = 2
xylims = [ -3 3 -3 3 ]
longtray = 5
fr = 500
ruta_gifs = joinpath( ruta_resultados * "gifs" );
mkpath( ruta_gifs );
cd( ruta_gifs )
for i in 1:cachos
    start = Int(( tamaño * ( i - 1) + 1 ))
    finish = Int(( tamaño * i ))
    limites = extrema(MVcsda[:,:, start:finish ])
    anima = @animate for i in start:finish
        A = heatmap(MVcsda[:,:,i] , clims=limites , yflip = true )
    end
    NomGifCSDA = string( nomcsda , lpad( i , nd , "0") , ".gif" )
    gif(anima, NomGifCSDA , fps = 120);
    anima = TraysFrameXFrame(longtray , start , finish , IniciosP , FinalesP , TraysPosP 
                            , CTP , xylims, IniciosN , FinalesN , TraysNegP , CTN , fr )
    NomBinTray = string( nombintray , lpad( i , nd , "0") , ".gif" )
    gif( anima, NomBinTray , fps = 120);
end

[ Info: Saved animation to e:\reyal\Documents\gitWorkspace\cinvestav\EEG\exp136_Igancio_32CH_Resultadosgifs\CSDA_01.gif
[ Info: Saved animation to e:\reyal\Documents\gitWorkspace\cinvestav\EEG\exp136_Igancio_32CH_Resultadosgifs\Trayectorias_01.gif
[ Info: Saved animation to e:\reyal\Documents\gitWorkspace\cinvestav\EEG\exp136_Igancio_32CH_Resultadosgifs\CSDA_02.gif
[ Info: Saved animation to e:\reyal\Documents\gitWorkspace\cinvestav\EEG\exp136_Igancio_32CH_Resultadosgifs\Trayectorias_02.gif
[ Info: Saved animation to e:\reyal\Documents\gitWorkspace\cinvestav\EEG\exp136_Igancio_32CH_Resultadosgifs\CSDA_03.gif
[ Info: Saved animation to e:\reyal\Documents\gitWorkspace\cinvestav\EEG\exp136_Igancio_32CH_Resultadosgifs\Trayectorias_03.gif
[ Info: Saved animation to e:\reyal\Documents\gitWorkspace\cinvestav\EEG\exp136_Igancio_32CH_Resultadosgifs\CSDA_04.gif
[ Info: Saved animation to e:\reyal\Documents\gitWorkspace\cinvestav\EEG\exp136_Igancio_32CH_Resultadosgifs\Trayectorias_04.gif
[ Info: 

In [9]:
# Ahora el proposito es generar una matriz donde se diga con qué elementos está conectado
# y lograr graficar la estructura



# Conexiones = MatrizDeConexiones( Headers , Coordenadas , Prohibidas , combs );


# Esta es una prueba de una matriz interesante para representar el voltaje con Conexiones
n = 62
ImageConexiones = graphplot(Conexiones,
          markersize = 0.17,
          #node_weights = MV[1],
          curves = false,
          background_color = :darkgray,
          markercolor = range(colorant"yellow", stop=colorant"red", length=n),
          names = Headers,
          fontsize = 10,
          nodeshape = :circle,
          linecolor = :darkgrey
          );




# Se crea un array MV ideal para graficar en la matriz de tipo conexiones y Cuadrada
MV = ArrayDeConexiones( Canales , Headers );
NomConex = Archivo*"_DatosEnConexiones.jld"
ruta_archivo = joinpath( ruta_resultados , NomConex )
#save( ruta_archivo , "MV" , MV )

LoadError: UndefVarError: Conexiones not defined

In [ ]:
CatenarioPositivo , CatenarioNegativo = Trayectorias( CMP , CMN , 1000 );

In [ ]:
TraysPos = Dinamica_LV( CMP , DistTol , TTol );
TraysNeg = Dinamica_LV( CMN , DistTol , TTol );